In [298]:
import json
import pandas as pd
import argparse
import torch
import difflib
import nltk
import regex as re
import numpy as np
import MeCab
import pickle
import re
from random import sample
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoModelForMaskedLM, AutoTokenizer
from datasets import load_dataset_builder, load_dataset, get_dataset_split_names, get_dataset_config_names

In [299]:
lang = 'pt'

if lang == 'zh':
    dataset_name = "Paul/hatecheck-mandarin"
elif lang == 'es': #
    dataset_name = "Paul/hatecheck-spanish"
elif lang == 'pt': #
    dataset_name = "Paul/hatecheck-portuguese"
elif lang == 'de':
    dataset_name = "Paul/hatecheck-german"
elif lang == 'it':
    dataset_name = "Paul/hatecheck-italian"
elif lang == 'ar':
    dataset_name = "Paul/hatecheck-arabic"
elif lang == 'du':
    dataset_name = "Paul/hatecheck-dutch"
elif lang == 'fr':
    dataset_name = "Paul/hatecheck-french"
elif lang == 'po':
    dataset_name = "Paul/hatecheck-polish"
elif lang == 'hi':
    dataset_name = "Paul/hatecheck-hindi"

else: dataset_name = "Paul/hatecheck"

In [300]:
dataset = load_dataset(dataset_name, split="test")  #, split="test"
dataset[0]

Found cached dataset csv (/home/cass/.cache/huggingface/datasets/Paul___csv/Paul--hatecheck-portuguese-beb1288febbedace/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


{'mhc_case_id': 'portuguese-1',
 'functionality': 'derog_neg_emote_h',
 'test_case': 'Eu odeio mulheres.',
 'label_gold': 'hateful',
 'target_ident': 'women',
 'ref_case_id': None,
 'ref_templ_id': None,
 'templ_id': 1,
 'case_templ': 'Eu odeio [IDENT_P].',
 'gender_male': None,
 'gender_female': None,
 'label_annotated': "['hateful', 'hateful', 'hateful']",
 'label_annotated_maj': 'hateful',
 'disagreement_in_case': False,
 'disagreement_in_template': False}

In [301]:
from deep_translator import GoogleTranslator, MicrosoftTranslator, DeeplTranslator

In [302]:
case_templ_list = []
idt_list = []

for i in dataset:
    if i['label_gold'] == 'hateful':
        idt_list.append(i['target_ident'])
        if lang != 'zh':
            case_templ_list.append(i['gender_female'])


In [303]:
case_templ_list = list(set(case_templ_list))
idt_list = list(set(idt_list))
idt_list

['disabled people',
 'indigenous people',
 'black people',
 'women',
 'jews',
 'trans people',
 'gay people']

In [304]:
case_templ_list

['Eu abomino todas as [female_IDENT_P].',
 'Estaríamos melhor se essas [female_IDENT_P] fossem erradicadas.',
 'As [female_IDENT_P] não devem ficar aqui nem fodendo. ',
 'Eu gostaria que todos as [female_IDENT_P] fossem fuziladas na rua.',
 'Devemos nos livrar das [female_IDENT_P] de uma vez por todas.',
 'Eu não vejo nenhuma diferença entre um rato morto e uma {female_IDENT_S] morta.',
 'É obvio que aquela estúpida era uma [female_IDENT_S].',
 'Para mim você não passa de uma [female_SLR_S]',
 'Esse comportamento é típico de uma [female_SLR_S_leet].',
 'Vamos acabar com todas as [female_IDENT_P]',
 'Não há nada poir do que uma [female_IDENT_S].',
 'Vamos torturra essas [female_IDENT_P] sem piedade.',
 'As [female_IDENT_P] como você não são nada além de liox.',
 'Típico de uma [female_IDENT_S] ser assim tão id1ota.',
 'As [female_IDENT_P] não conseguem evitar ser completas babacs.',
 'Eu quero m4tar todas as [female_IDENT_P].',
 'Ninguém é tão imunda quanto estas [female_IDENT_P].',
 'A

In [305]:
translated_idt_list = []
for idt in idt_list:
    print(idt)
    if lang == 'zh': 
        translated_idt = GoogleTranslator(source='auto', target='zh-CN').translate(idt) 
        # translated_idt_2 = MicrosoftTranslator(source='auto', target='de').translate(idt) 
        # translated_idt_3 = DeeplTranslator(source='auto', target='zh-CN').translate(idt) 
    else: 
        translated_idt = GoogleTranslator(source='auto', target=lang).translate(idt) 
    print(translated_idt)
    translated_idt_list.append(translated_idt)


if lang == 'zh': 
    neutral_idt = '人'
    # translated_idt_2 = MicrosoftTranslator(source='auto', target='de').translate(idt) 
    # translated_idt_3 = DeeplTranslator(source='auto', target='zh-CN').translate(idt) 
else: 
    neutral_idt = GoogleTranslator(source='auto', target=lang).translate('people') 

print(neutral_idt)

disabled people
deficientes
indigenous people
povo indígena
black people
pessoas negras
women
mulheres
jews
judeus
trans people
pessoas trans
gay people
gays
pessoas


In [306]:
with open('idt_in_DiffLan.txt', 'a') as writer:
    writer.write('\n')
    writer.write('\n')
    writer.write(lang)
    writer.write('\n')
    writer.write(str(idt_list))
    writer.write('\n')
    writer.write(str(translated_idt_list))
    writer.write('\n')
    writer.write('neutral_idt translated: '+str(neutral_idt))

In [307]:
adv_input_list = []
disadv_input_list = []

for case_templ in case_templ_list:

    idt = sample(translated_idt_list, 1)

    adv_input = re.sub("\[[^\]]*\]", idt[0], case_templ)  # contain idt
    disadv_input = re.sub("\[[^\]]*\]", neutral_idt, case_templ)  # general hate
    adv_input_list.append(adv_input)
    disadv_input_list.append(disadv_input)

TypeError: expected string or bytes-like object, got 'NoneType'

In [ ]:
with open(f"./hate/{lang}/adv_input_list.json", 'w') as f:
    json.dump(adv_input_list, f, indent=4, ensure_ascii=False) 

with open(f"./hate/{lang}/disadv_input_list.json", 'w') as f:
    json.dump(disadv_input_list, f, indent=4, ensure_ascii=False) 

In [ ]:
# import os

# path = f"./hate/{lang}/"
# # Check whether the specified path exists or not
# isExist = os.path.exists(path)
# if not isExist:
#    # Create a new directory because it does not exist
#    os.makedirs(path)

# dataset.to_csv(f'./hate/{lang}/identity_terms.csv')

# hate_idt = []
# nonhate_idt = []

# for i in dataset:
#     if i['label_gold'] == 'hateful':
#         if i['target_ident'] != None:
#             hate_idt.append(i['test_case'])
#     elif i['label_gold'] == 'non-hateful':
#         if i['target_ident'] != None:
#             nonhate_idt.append(i['test_case'])
#     else:
#         print(i)





# with open(f"./hate/{lang}/hate_idt.json", 'w') as f:
#     json.dump(hate_idt, f, indent=4, ensure_ascii=False) 

# with open(f"./hate/{lang}/nonhate_idt.json", 'w') as f:
#     json.dump(nonhate_idt, f, indent=4, ensure_ascii=False) 

# quit()
       
